In [1]:
import pandas as pd
import glob


In [2]:
# ### Add missing Cargo ID when it is not present in the CSV file

# mapping = {
#     "DIPUTADOS NACIONALES": 3,
#     "DIPUTADOS PROVINCIALES": 6,
#     "CONCEJALES": 10,
#     "SENADORES PROVINCIALES": 5,
#     "SENADORES NACIONALES": 2,
#     "INTENDENTES": 7,
#     "DIPUTADO NACIONAL": 3,
#     "SENADOR NACIONAL": 2,
#     "DIPUTADO PROVINCIAL": 6,
#     "SENADOR PROVINCIAL": 5
# }

# filepaths = [    "./../datos/Solo_ResultadosProvisorios_bkp/ResultadosElectorales_General2021/ResultadosElectorales.csv",
#                  "./../datos/Solo_ResultadosProvisorios_bkp/ResultadosElectorales_PASO2021/ResultadosElectorales.csv",
#                          "./../datos/Solo_ResultadosProvisorios_bkp/ResultadosElectorales_General2017/ResultadosElectorales.csv",]

# for filepath in filepaths:
#     print(filepath)
#     with open(filepath, "r") as file:
#         df = pd.read_csv(file)
#     # print(df["cargo_nombre"].value_counts())
#     df["cargo_id"] = df["cargo_nombre"].map(mapping)
#     display(df["cargo_id"].fillna(-1).value_counts())
#     display(df["cargo_nombre"].fillna(-1).value_counts())
#     with open(filepath.replace("/Solo_ResultadosProvisorios_bkp/", "/Solo_ResultadosProvisorios/"), "w") as file:
#         df.to_csv(file, index=False)


In [3]:
# Define the glob pattern to find the CSV files
csv_votos_pattern = './../datos/Solo_ResultadosProvisorios/**/*.csv'

# Find all the CSV files that match the glob pattern
csv_votos_files = glob.glob(csv_votos_pattern, recursive=True)

In [4]:
# Read all the CSV files into a list of DataFrames, lowercase their columns, and concatenate them
df_list = []
for file in csv_votos_files:
    df = pd.read_csv(file, dtype=object)
    df = df.rename(columns=lambda x: x.lower())
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
# 7 minutes

In [5]:
# data.columns

# Index(['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo', 'distrito_id',
#        'distrito_nombre', 'seccionprovincial_id', 'seccionprovincial_nombre',
#        'seccion_id', 'seccion_nombre', 'circuito_id', 'circuito_nombre',
#        'mesa_id', 'mesa_electores', 'mesa_tipo', 'cargo_id', 'cargo_nombre',
#        'agrupacion_id', 'agrupacion_nombre', 'lista_numero', 'lista_nombre',
#        'votos_tipo', 'votos_cantidad'],
#       dtype='object')

In [6]:
# data.to_csv('./full_data_bkp.csv', index=False)

# data = pd.read_csv('./full_data_bkp.csv') conviene correr el loop y generarlo.

In [7]:
### Clean the data and make necessary corrections to improve data quality (fillna, typecasting, etc.)

# Fill missing values in seccionprovincial_id with 0
data['seccionprovincial_id'].fillna(0, inplace=True)

# Convert distrito_id to int
data['distrito_id'] = data['distrito_id'].astype(int)

# Convention, fill with zeros to make circuit id 6 digits
data['circuito_id'] = data['circuito_id'].str.zfill(6)

# Convert seccion_id to int and change case of seccion_nombre to title case
data['seccion_id'] = data['seccion_id'].astype(int)
data['seccion_nombre'] = data['seccion_nombre'].fillna('')

In [8]:
# Generate an id for eleccion
gr_eleccion = ['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo']
data['eleccion_id'] = data.groupby(gr_eleccion).ngroup()

# Casi 10 mins

In [9]:
## Harmonize 'votos_tipo' column
import json

with open('./../datos/correccion/valor_votos_tipo.json', 'r') as f:
    valor_votos_tipo_homog = json.load(f)

data['votos_tipo'] = data['votos_tipo'].map(valor_votos_tipo_homog)

## Decomposition into relational tables

In [10]:
def decompose_data(data, group_cols, table_cols, table_name):
    """
    Creates a table with unique values from the specified columns of the given data, grouped by the specified columns.
    Saves the resulting table as a CSV with the specified name.
    Removes the specified columns from the original data.
    
    Args:
        data (pd.DataFrame): Input DataFrame
        group_cols (list): List of columns to group by
        table_cols (list): List of columns to include in resulting table
        table_name (str): Name of table to save
    
    Returns:
        None
    """

    # Create table with unique values
    table_df = data[group_cols + table_cols].drop_duplicates().reset_index(drop=True)
    
    # Save table as CSV
    table_df.to_csv('./../datos/BD/' + table_name + "_table.csv", index=False)
    
    # Remove table_cols from original data
    data.drop(table_cols, axis=1, inplace=True)


In [11]:
data_bkp = data.copy() # In case we make a mistake or wish to change something in the following cells without running from the start

In [ ]:
# Create a table for eleccion
decompose_data(data, ['eleccion_id'], ['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo'], 'eleccion')

In [ ]:

# Create a table for seccion
decompose_data(data, ['distrito_id', 'seccion_id', 'seccionprovincial_id'], ['seccion_nombre'], 'seccion')

# Create a table for seccionprovincial
decompose_data(data, ['distrito_id', 'seccionprovincial_id'], ['seccionprovincial_nombre'], 'seccionprovincial')

# Create a table for distrito
decompose_data(data, ['distrito_id'], ['distrito_nombre'], 'distrito')

# Create a table for mesas
decompose_data(data, ['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id'], ['mesa_electores', 'mesa_tipo'], 'mesas')

# Create a table for cargo
decompose_data(data, ['cargo_id'], ['cargo_nombre'], 'cargo')

# Create a table for agrupacion
decompose_data(data, ['eleccion_id', 'distrito_id', 'agrupacion_id', 'votos_tipo', 'lista_numero'], ['lista_nombre'], 'agrupacion_lista')  # lista nombre no deberia quedar porque puede ser mismo numero con distinto lista nombre

In [ ]:
## 
# Cargo, revisar que el 7 deberia ser intendente y sale concejal

In [ ]:
# Create a table for circuitos. # Se admite que cambien de eleccion a eleccion
decompose_data(data, ['eleccion_id', 'distrito_id', 'seccion_id', 'seccionprovincial_id', 'circuito_id'], ['circuito_nombre'], 'circuito')

In [ ]:
data.head()

,distrito_id,seccionprovincial_id,seccion_id,circuito_id,mesa_id,cargo_id,agrupacion_id,lista_numero,votos_tipo,votos_cantidad,eleccion_id
0,1,0,15,000161,6883,3,509.0,3072.0,POSITIVO,6,1
1,1,0,15,000161,6901,3,509.0,3072.0,POSITIVO,14,1
2,1,0,15,000161,6914,3,509.0,3072.0,POSITIVO,8,1
3,1,0,15,000159,6813,3,509.0,3074.0,POSITIVO,0,1
4,1,0,15,000161,6883,3,509.0,3074.0,POSITIVO,6,1


## Harmonize names of Cargo, Distrito, Seccion.

In [ ]:
def names_harmonization(df, id_cols, name_col):
    df[name_col] = df[name_col].str.title() ###
    df[name_col] = df.dropna().groupby(id_cols)[name_col].transform(lambda x: x.mode()[0])
    df.drop_duplicates(subset=id_cols, inplace=True)
    return df

# Load the seccion table and apply name harmonization
seccion_df = pd.read_csv('./../datos/BD/seccion_table.csv'); print(seccion_df.shape)
seccion_df = names_harmonization(seccion_df, ['distrito_id', 'seccion_id'], 'seccion_nombre')
seccion_df.to_csv('./../datos/BD/seccion_table.csv', index=False)

# Load the cargo table and apply name harmonization
cargo_df = pd.read_csv('./../datos/BD/cargo_table.csv'); print(cargo_df.shape)
cargo_df = names_harmonization(cargo_df, ['cargo_id'], 'cargo_nombre')
cargo_df.to_csv('./../datos/BD/cargo_table.csv', index=False)

# Load the distrito table and apply name harmonization
distrito_df = pd.read_csv('./../datos/BD/distrito_table.csv'); print(distrito_df.shape)
distrito_df = names_harmonization(distrito_df, ['distrito_id'], 'distrito_nombre')
distrito_df.to_csv('./../datos/BD/distrito_table.csv', index=False)

# Load the mesas table and apply name harmonization
df_mesas = pd.read_csv('./../datos/BD/mesas_table.csv')
df_mesas['mesa_tipo'] = df_mesas['mesa_tipo'].replace('NATIVO', 'NATIVOS').replace('EXTRANJERO', 'EXTRANJEROS')
df_mesas = df_mesas.drop_duplicates()
df_mesas.to_csv('./../datos/BD/mesas_table.csv', index=False)


(1198, 4)
(15, 2)
(51, 2)


## Save Votos datasets

In [ ]:
for eleccion_id, df in data.groupby('eleccion_id'):
    filename = f'./../datos/BD/votos_eleccion_{eleccion_id}_table.csv'
    df = df.drop_duplicates() # Especially because of 2021
    print(df.shape)
    df.to_csv(filename, index=False)

(3503909, 11)
(3503909, 11)
(2116876, 11)
(2116876, 11)
(1697141, 11)
(1697141, 11)
(2277250, 11)
(2277250, 11)
(565020, 11)
(565020, 11)
(5053072, 11)
(5053072, 11)
(8956666, 11)
(8956666, 11)
(204116, 11)
(204116, 11)
(2444966, 11)
(2438892, 11)
(153773, 11)
(153773, 11)
(4807171, 11)
(4807171, 11)
(264145, 11)
(264145, 11)
(3159297, 11)
(3159297, 11)
(303176, 11)
(303176, 11)
(5082019, 11)
(5082019, 11)
(461434, 11)
(230717, 11)
(10428980, 11)
(5214490, 11)


In [ ]:
count_check = data.groupby('eleccion_id').count()

In [ ]:
count_check.div(count_check.max(1), 0) ## Los faltantes en agrupacion id y lista id tienen origen en la fuente de datos
## Los numeros de lista son para elecciones primarias, no para elecciones generales.
## Ademas, no hay lista o agrupacion para votos no positivos (blancos, nulos, recurridos, etc.)

,distrito_id,seccionprovincial_id,seccion_id,circuito_id,mesa_id,cargo_id,agrupacion_id,lista_numero,votos_tipo,votos_cantidad
eleccion_id,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,0.640411,0.001853,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,0.736808,0.119236,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,0.606258,0.000000,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.677434,0.677434,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,0.000000,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,0.574375,0.003887,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,0.763525,0.726825,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,0.610419,0.000000,1.0,1.0


In [ ]:
# for file in csv_votos_files[:4]:
#     print(file)
#     df = pd.read_csv(file, usecols = ['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo', 'agrupacion_id', 'lista_numero', 'agrupacion_nombre', 'lista_nombre'])
#     print(df.dtypes)
#     print(df.count())
#     # Create table with unique values
#     table_df = df[['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo', 'agrupacion_id', 'lista_numero'] + ['agrupacion_nombre', 'lista_nombre']].drop_duplicates().reset_index(drop=True)
    
#     print(table_df.dtypes)

#     display(pd.Series(df.agrupacion_id.unique()).sample(50).values)
#     display(pd.Series(table_df.agrupacion_id.unique()).sample(50).values)
#     print('########################################')